In [1]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, f1_score, roc_auc_score, accuracy_score

import numpy as np
import utils as ut
import pandas as pd

[nltk_data] Downloading package punkt to
[nltk_data]     /users/Etu7/21200397/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /users/Etu7/21200397/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /users/Etu7/21200397/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /users/Etu7/21200397/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/tmp/ipykernel_16209/1321755799.py:12: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems

In [2]:
import warnings

# Ignorer tous les avertissements
warnings.filterwarnings("ignore")

# Ignorer les avertissements spécifiques par leur catégorie
warnings.filterwarnings("ignore", category=UserWarning)


# Etape 0 : Chargement de données movies

Dans cette partie nous allons rechargé les données de movies afin d'effectuer l'analyse de sentiments

In [3]:
path = "../datasets/movies/movies1000/"
alltxts,alllabs = ut.load_movies(path)

Création du DataFrame

In [4]:
movies_df = pd.DataFrame()
movies_df['text'] = alltxts
movies_df['label'] = alllabs

movies_df

,text,label
0,supposedly based on a true story in which the ...,0
1,one might expect a cathartic viewing experienc...,0
2,the camera zooms in incredibly close . \nit fo...,0
3,1990s would remembered as the era of binary mo...,0
4,""" the world on land -- it's just too big for ...",0
...,...,...
1995,one of the more unusual and suggestively viole...,1
1996,i find most of television so intensely boring ...,1
1997,the event of events is upon us . \npeople have...,1
1998,the sheer horrific audacity of the nazi plan t...,1


# Grid search pour TF-IDF Vectorizer

In [5]:

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')),
    ('reg', LogisticRegression())
])

parameters = {
    'tfidf__max_df': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    'tfidf__min_df': [2, 3, 5, 10],
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3), (2, 2), (2, 3) , (3,3)],
    'tfidf__binary': [False, True],
    'tfidf__lowercase': [False, True],
    'tfidf__use_idf': [False, True],
    'tfidf__sublinear_tf': [False, True],
    'tfidf__max_features': [None, 2000, 5000, 10000, 20000 , 40000],

    'reg__C': [0.1, 1, 10, 100],
    'reg__penalty': ['none', 'l2' , 'l1']
}

scoring = {
    'f1_score': make_scorer(f1_score),
    'roc_auc': make_scorer(roc_auc_score),
    'accuracy': make_scorer(accuracy_score)
}

[X_all_train, X_all_test, Y_train, y_test]  = train_test_split(alltxts, alllabs, test_size=0.2, random_state=10, shuffle=True)

grid_search = GridSearchCV(pipeline, parameters,scoring=scoring, refit='f1_score', cv=5, n_jobs=-1, verbose=1)


grid_search.fit(X_all_train, Y_train)


print("Meilleurs paramètres trouvés:")
print(grid_search.best_params_)


print("Scores:")
print("F1 Score:", grid_search.cv_results_['mean_test_f1_score'])
print("AUC:", grid_search.cv_results_['mean_test_roc_auc'])
print("Accuracy:", grid_search.cv_results_['mean_test_accuracy'])

Fitting 5 folds for each of 165888 candidates, totalling 829440 fits


ValueError: 
All the 829440 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
51842 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 890, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1351, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py", line 471, in fit
    Xt = self._fit(X, y, routed_params)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py", line 408, in _fit
    X, fitted_transformer = fi

# Grid search pour la régression logistic

In [8]:
f = lambda doc: ut.suppression_balises_html( ut.remove_numbers(ut.remove_ponctuation(doc)) ) 

In [15]:
lr_grid_params = {'penalty': ['l1','l2' , None],
          'C': [0.001, 0.01, 0.1, 1, 10, 100]}

tf_idf_vectorizer = TfidfVectorizer(preprocessor=f, ngram_range=(1, 3), stop_words='english',tokenizer=word_tokenize, lowercase=True, max_features=40000)
[X_all_train, X_all_test, Y_train, y_test]  = train_test_split(alltxts, alllabs, test_size=0.2, random_state=10, shuffle=True)
X_train = tf_idf_vectorizer.fit_transform(X_all_train)
X_test = tf_idf_vectorizer.transform(X_all_test)

lr_clf = LogisticRegression(max_iter=100)


lr_grid_params = {'penalty': ['l1','l2' , None],
          'C': [0.001, 0.01, 0.1, 1, 10, 100]}

scoring = {
    'f1_score': make_scorer(f1_score, average='macro'),  # F1 score
    'roc_auc': make_scorer(roc_auc_score, needs_proba=True, multi_class='ovr'),  # AUC
    'accuracy': make_scorer(accuracy_score)  # Accuracy
}

lr_grid_search = GridSearchCV(lr_clf, param_grid=lr_grid_params, cv=5 , scoring=scoring, refit='f1_score')

lr_grid_search.fit(X_train, Y_train)

print('Meilleurs hyperparamètres trouvés : ', lr_grid_search.best_params_)
print()
# Affichage des scores moyens pour chaque métrique
results = lr_grid_search.cv_results_
print("Scores moyens pour chaque métrique :")
for metric in ['f1_score', 'roc_auc', 'accuracy']:
    print(f"{metric}: {results[f'mean_test_{metric}']}")

Meilleurs hyperparamètres trouvés :  {'C': 100, 'penalty': 'l2'}

Scores moyens pour chaque métrique :
f1_score: [       nan 0.33471933 0.83159619        nan 0.42948704 0.83159619
        nan 0.78704815 0.83159619        nan 0.8192636  0.83159619
        nan 0.83730742 0.83159619        nan 0.83981059 0.83159619]
roc_auc: [       nan 0.8709559  0.90873862        nan 0.87145592 0.90873862
        nan 0.87513575 0.90873862        nan 0.89444119 0.90873862
        nan 0.91209032 0.90873862        nan 0.91563733 0.90873862]
accuracy: [     nan 0.503125 0.831875      nan 0.546875 0.831875      nan 0.788125
 0.831875      nan 0.819375 0.831875      nan 0.8375   0.831875      nan
 0.84     0.831875]
